# Description
This notebook test the configuration to start Pyspark 3.x.

In [1]:
import os

In [2]:
from pyspark.sql import SparkSession
import configparser, os

In [3]:
def init_spark(core=3, memory='4g'):
    # Init spark
    spark = (SparkSession.builder
             .config('spark.master', 'local[%d]' % core) # Use 3 cores -- leave one core for others.
             .config('spark.driver.memory', memory)
             .config('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.1.2')
             .appName("Standard Pyspark").getOrCreate()
            )
    # Setup credentials.
    # Assume the credientials is locasted at ~/.aws 
    aws_profile = 'default'
    config = configparser.ConfigParser()
    config.read(os.path.expanduser("~/.aws/credentials"))
    access_id = config.get(aws_profile, "aws_access_key_id")
    access_key = config.get(aws_profile, "aws_secret_access_key")
    
    # Authenticate Hadoop (s3)
    hadoop_conf = spark.sparkContext._jsc.hadoopConfiguration()
    hadoop_conf.set("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    hadoop_conf.set("fs.s3a.access.key", access_id)
    hadoop_conf.set("fs.s3a.secret.key", access_key)
    
    return spark

In [4]:
spark = init_spark()

:: loading settings :: url = jar:file:/home/ds_user/miniconda/envs/datascience/lib/python3.8/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ds_user/.ivy2/cache
The jars for the packages stored in: /home/ds_user/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-eab899d9-d5f7-4010-88a1-935980072950;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.1.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.271 in central
:: resolution report :: resolve 200ms :: artifacts dl 6ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.271 from central in [default]
	org.apache.hadoop#hadoop-aws;3.1.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	------------------------------

In [7]:
spark

In [5]:
business_tab = spark.read.parquet('s3a://adc-ds-lms/aspirasi/prod/lms/adc_lms/latest_data/business_tab.parquet/')


21/10/12 01:30:14 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [11]:
business_tab.select('bt_state').show(3)

+--------------------+
|            bt_state|
+--------------------+
|Daerah Khusus Ibu...|
|             Cirebon|
|             Cirebon|
+--------------------+
only showing top 3 rows

